In [2]:
import os.path
import urllib
import zipfile
import glob
import operator

import pymysql
import pandas as pd
from sqlalchemy import create_engine

In [61]:
from newspaper import Article
import re
from sklearn.cross_validation import train_test_split

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Read Training and Test Data

In [47]:
training_data_file = open('training_fix.csv', "r") 
test_data_file = open('test_fix.csv', "r") 

training_data = training_data_file.readlines()
test_data = test_data_file.readlines()

## Sentiment Analysis

In [57]:
corpus = []
for p in training_data:
    d = p.split(';')
    corpus.append({'text':d[0],'sentiment':d[1].replace('\n','')})

In [58]:
news_corpus = pd.DataFrame.from_dict(corpus)

In [59]:
X = news_corpus.text
y = news_corpus.sentiment
print(X.shape)
print(y.shape)

(80,)
(80,)


In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(72,)
(8,)
(72,)
(8,)


In [64]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(X_train)


In [65]:
X_train_dtm = vect.transform(X_train)
X_test_dtm = vect.transform(X_test)

## SVM

In [66]:
from sklearn import svm
from sklearn.metrics import accuracy_score
clf = svm.SVC(kernel='linear')

In [67]:
%time clf.fit(X_train_dtm, y_train)

Wall time: 18 ms


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [68]:
print("Predicting...")
prediction = clf.predict(X_test_dtm)

Predicting...


In [69]:
print("Prediction:",prediction)
accuracy = accuracy_score(y_test, prediction)
print('Accuracy:', accuracy)

Prediction: ['-1' '1' '1' '1' '1' '1' '-1' '-1']


In [83]:
new_article = ['Read on to see how.']
new_article_vect = vect.transform(new_article)
clf.predict(new_article_vect)

array(['-1'], dtype=object)

## Uji coba pada data test

In [91]:
result = []
for t in test_data:
    new_article = [t]
    new_article_vect = vect.transform(new_article)
    label = clf.predict(new_article_vect)
    result.append({'article':t, 'label':int(label)})
    
pd.DataFrame.from_dict(result)

,label,text
0,-1,Jong-nam trial: Doan had no idea ‘oily liquid’...
1,-1,Bali`s Tourism Remains Unaffected by Active Mo...
2,1,Indonesia: Activists Protest Criminal Code Rev...
3,-1,Indonesia Commend Timor Leste and Australia Se...
4,1,How Google Maps Typos Grow Into Real Neighborh...
5,-1,Gerindra`s Chance to Back Prabowo-Anies Baswed...
6,-1,Dr M: US confirms Equanimity bought by Jho Low...
7,1,Situation Precarious For Tens Of Thousands Of ...
8,-1,"""We asked our readers what we can do to help d..."
9,1,Indonesian president defends death penalty for...
